In [ ]:
!pip install tensorflow_addons

# Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow import keras
import tensorflow
from time import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D

# Loading the Data

In [ ]:
features_train = np.load("../input/data-xmer/features_trainx.npy")
target_train = np.load("../input/data-xmer/target_trainoex.npy")
features_test = np.load("../input/data-xmer/features_testx.npy")
target_test = np.load("../input/data-xmer/target_testoex.npy")
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

In [ ]:
target_train.shape

# Building the Model Architecture

In [ ]:
def Encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multihead-Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Network
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
n_classes = 3
def Model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = Encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = layers.Dense(30, activation = 'relu')(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [ ]:
input_shape = features_train.shape[1:]

model = Model(
    input_shape,
    head_size=200,
    num_heads=2,
    ff_dim=4,
    num_transformer_blocks=2,
    mlp_units=[56],
    mlp_dropout=0.3, #0.4
    dropout=0.2,#0.255
)

model.summary()

In [ ]:
tf.keras.utils.plot_model(model,to_file = 'xmer_1.png', show_shapes=True)

In [ ]:
lr = 1e-3

optimizer = tfa.optimizers.RectifiedAdam(learning_rate = lr)

model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(), 
              metrics = 'accuracy')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                 factor = 0.2,
                                                 patience = 1,
                                                 verbose = 2,
                                                 min_delta = 1e-4,
                                                 min_lr = 1e-9,
                                                 mode = 'max')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 3,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath ='./xmer_1.hdf5',
                                                  monitor = 'val_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')


class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time() - self.epoch_time_start)
        

time_callback = TimeHistory()

callbacks = [earlystopping, reduce_lr, checkpointer , time_callback] 

# Training the model

In [ ]:
int_epoch = 10
batch = 16
cw = {0:0,1:0,2:0.6}
history = model.fit(
    features_train,
    target_train,
    validation_data = (features_test, target_test),
    epochs=int_epoch,
    batch_size= batch,
    callbacks=callbacks,
    class_weight = cw
     )

In [ ]:
 print(f"Loss: {history.history['loss'][-1]}")       
print(f"accuracy: {history.history['accuracy'][-1]}")   
print(f"Validation accuracy: {history.history['val_accuracy'][-1]}")  
print(f"Training Time: {sum(time_callback.times)}")      

# Evaluating the model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss'] 


plt.figure(figsize=(9, 9)) 
plt.subplot(2, 1, 1)
plt.tight_layout(pad=3.0)
plt.plot(acc, label='Train' , linewidth = 3)
plt.plot(val_acc, label='Validation', linewidth = 3)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(loc='upper left' , prop={'size': 16})
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Accuracy' , fontsize=16)
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation accuracy', fontsize=16)

plt.subplot(2, 1, 2)
plt.tight_layout(pad=3.0)
plt.plot(loss, label='Train', linewidth = 3)
plt.plot(val_loss, label='Validation', linewidth = 3)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(loc='center left', prop={'size': 16})
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Cross-entropy Loss', fontsize=16)
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Cross-entropy Loss', fontsize=16)
plt.show()

In [ ]:
loss = np.array(loss)
val_loss = np.array(val_loss)
acc = np.array(acc)
val_acc = np.array(val_acc)

np.save("./xmer_1loss.npy", loss) 
np.save("./xmer_1val_loss.npy", val_loss) 
np.save("./xmer_1acc.npy", acc) 
np.save("./xmer_1val_acc.npy", val_acc) 

In [ ]:
def load_trained_model(weights_path):
   model = build_model(
    input_shape,
    head_size=200,
    num_heads=2,
    ff_dim=4,
    num_transformer_blocks=2,
    mlp_units=[56],
    mlp_dropout=0.3,
    dropout=0.25,
)
   model.load_weights(weights_path)
   return model

new_model = load_trained_model("./xmer_1.hdf5")
new_model.summary()

In [ ]:
y_true = tf.argmax(target_test,axis = 1)
y_pred = new_model.predict(features_test)
y_predargmax = tf.argmax(y_pred , 1)
print(y_predargmax)

In [ ]:
from sklearn.metrics import classification_report
clreport = classification_report(y_true, y_predargmax , digits = 4)
print(clreport) 

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
classes = ['Baseline','Stress','Amusement']
matrix = confusion_matrix(y_true, y_predargmax, normalize= 'true')
plt.figure(figsize=(7, 7))
ax = sns.heatmap(matrix *100 ,
                cmap='coolwarm',
                linecolor='white',
                linewidths=1,
                annot=True,
                fmt='.2f')
for t in ax.texts: t.set_text(t.get_text() + " %")
plt.title('Confusion Matrix',fontsize = 16)
plt.ylabel('True Labels',fontsize = 15)
plt.xlabel('Predicted Labels', fontsize = 15)
plt.yticks(verticalalignment = 'center')
ax.xaxis.set_ticklabels(classes,fontsize = 13); ax.yaxis.set_ticklabels(classes,fontsize = 13)
plt.show()
